#Temporary example of selecting rows from the feature store, using pyodbc with SQL ODBC Driver 17

(Not for prod use, and odbc 18 should be used)

In [ ]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

In [ ]:
import pyodbc 
import sys

secret_scope = "flowehr-secrets"
server = dbutils.secrets.get(scope=secret_scope, key="flowehr-dbks-sql-server")
database = dbutils.secrets.get(scope=secret_scope, key="flowehr-dbks-sql-database")
client_id = dbutils.secrets.get(scope=secret_scope, key="flowehr-dbks-sql-app-id")
client_secret = dbutils.secrets.get(scope=secret_scope, key="flowehr-dbks-sql-app-secret")

def create_con_str(db: str) -> str:
    driver = "{ODBC Driver 17 for SQL Server}"
    return f'DRIVER={driver};SERVER={server};DATABASE={db};ENCRYPT=yes;Authentication=ActiveDirectoryServicePrincipal;UID={client_id};PWD={client_secret}'

# connect to database
cnxn = pyodbc.connect(create_con_str(database))
cursor = cnxn.cursor()

cursor.execute("SELECT top 10 * from dbo.diabetes") 

row = cursor.fetchone() 
while row: 
    display(row)
    row = cursor.fetchone()

